# Movie Recommender System  -- Collaborative FIltering

In [18]:
#importing Modules
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import pickle

### Loading Dataset

In [2]:
movies=pd.read_csv("movies.csv")
ratings=pd.read_csv("ratings.csv")

In [3]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [4]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


### Data PreProcessing

In [5]:
print(movies.shape,ratings.shape)

(9742, 3) (100836, 4)


In [6]:
print("Movies Details")
print(movies.isnull().sum(),end="\n-----------------\n")
print("Ratings Details")
print(ratings.isnull().sum(),end="\n-----------------\n")

Movies Details
movieId    0
title      0
genres     0
dtype: int64
-----------------
Ratings Details
userId       0
movieId      0
rating       0
timestamp    0
dtype: int64
-----------------


In [7]:
print(movies.duplicated().sum(),ratings.duplicated().sum())

0 0


### Creating Recommender Model

In [8]:
Movies_rating=movies.merge(ratings,on="movieId")
Movies_rating.drop(["genres","timestamp"],axis=1,inplace=True)

In [9]:
Critics=Movies_rating.groupby("userId").count()["rating"]>200
Critics_viewer=Critics[Critics].index

In [10]:
filtered_rating=Movies_rating[Movies_rating["userId"].isin(Critics_viewer)]

famous_movies=filtered_rating.groupby("title").count()["rating"]>=50
final_rating=filtered_rating[filtered_rating["title"].isin(famous_movies[famous_movies].index)]

In [11]:
final_rating

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
5,1,Toy Story (1995),18,3.5
6,1,Toy Story (1995),19,4.0
7,1,Toy Story (1995),21,3.5
15,1,Toy Story (1995),45,4.0
...,...,...,...,...
91489,79132,Inception (2010),567,3.0
91490,79132,Inception (2010),573,5.0
91494,79132,Inception (2010),596,4.0
91496,79132,Inception (2010),599,3.0


In [12]:
movie_df=final_rating.pivot_table(index="title",columns="userId",values="rating")
movie_df.fillna(0,inplace=True)
movie_df

userId,1,4,6,18,19,20,21,28,41,42,...,594,596,597,599,600,603,605,606,608,610
title,,,,,,,,,,,,,,,,,,,,,
2001: A Space Odyssey (1968),0.0,0.0,0.0,4.0,3.0,0.0,0.0,3.0,2.0,5.0,...,0.0,4.0,0.0,5.0,4.0,5.0,0.0,5.0,3.0,4.5
300 (2007),0.0,0.0,0.0,3.5,0.0,0.0,3.5,4.5,2.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,5.0,4.0
Ace Ventura: Pet Detective (1994),0.0,0.0,3.0,2.5,2.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,2.5,3.0,0.0,0.0,0.0,3.5,3.0
Airplane! (1980),0.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,5.0,...,0.0,3.5,5.0,3.5,3.5,0.0,5.0,0.0,0.0,3.5
Aladdin (1992),0.0,4.0,5.0,3.5,3.0,5.0,4.0,0.0,0.0,0.0,...,4.5,0.0,4.0,3.0,3.5,0.0,3.5,0.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Who Framed Roger Rabbit? (1988),5.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,4.0,...,0.0,0.0,5.0,3.5,3.0,3.0,0.0,3.0,3.0,0.0
Willy Wonka & the Chocolate Factory (1971),5.0,4.0,3.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,3.0,5.0,3.0,4.0,0.0,3.5,0.0
"Wizard of Oz, The (1939)",5.0,5.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,...,4.0,4.0,4.0,3.0,3.0,5.0,2.0,0.0,2.5,3.5


In [13]:
similarity_score=cosine_similarity(movie_df)

In [14]:
def recommend(movie_name):
    # index fetch
    index=np.where(movie_df.index==movie_name)[0][0]
    similar_items=sorted(list(enumerate(similarity_score[index])),key=lambda x:x[1],reverse=True)[1:6]

    for i in similar_items:
        print(movie_df.index[i[0]])

In [17]:
recommend("X-Men (2000)")

X2: X-Men United (2003)
Matrix, The (1999)
Spider-Man (2002)
Star Wars: Episode V - The Empire Strikes Back (1980)
Star Wars: Episode VI - Return of the Jedi (1983)


### Dumping Files to use in Streamlit Frontend

In [19]:
pickle.dump(similarity_score,open("similarity.pkl","wb"))